In [2]:
import numpy as np
from scipy.sparse.linalg import spsolve
import scipy.sparse as sps
import sympy
from math import pi
from time import time

import pyamg

import setup_grids

from porepy.numerics.fv import mpsa, fvutils

from porepy.params.tensor import FourthOrderTensor as stiffness
from porepy.params import bc
from porepy.viz.plot_grid import plot_grid

In [3]:
# Permeability tensor, scalar for simplicity
mu = 1
lmbda = 1

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 4
grid_type = 'cart'

# Analytical solution
x, y = sympy.symbols('x y')
ux = sympy.sin(x) * sympy.cos(y)
uy = sympy.sin(x) * x**2
ux_f = sympy.lambdify((x, y), ux, 'numpy')
uy_f = sympy.lambdify((x, y), uy, 'numpy')
dux_x = sympy.diff(ux, x)
dux_y = sympy.diff(ux, y)
duy_x = sympy.diff(uy, x)
duy_y = sympy.diff(uy, y)
divu = dux_x + duy_y

sxx = 2 * mu * dux_x + lmbda * divu
sxy = mu * (dux_y + duy_x)
syx = mu * (duy_x + dux_y)
syy = 2 * mu * duy_y + lmbda * divu

sxx_f = sympy.lambdify((x, y), sxx, 'numpy')
sxy_f = sympy.lambdify((x, y), sxy, 'numpy')
syx_f = sympy.lambdify((x, y), syx, 'numpy')
syy_f = sympy.lambdify((x, y), syy, 'numpy')

rhs_x = sympy.diff(sxx, x) + sympy.diff(syx, y)
rhs_y = sympy.diff(sxy, x) + sympy.diff(syy, y)
rhs_x_f = sympy.lambdify((x, y), rhs_x, 'numpy')
rhs_y_f = sympy.lambdify((x, y), rhs_y, 'numpy')

def run_convergence(grid_type, pert):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert, domain=domain):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
            
        # Permeability tensor
        mu_c = mu * np.ones(g.num_cells)
        lmbda_c = lmbda * np.ones(g.num_cells)
        k = stiffness(2, mu_c, lmbda_c)

        # Set type of boundary conditions - Dirichlet
        bound_faces = g.get_all_boundary_faces()
        bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)
        
        # MPFA discretization, and system matrix
        stress, bound_stress = mpsa.mpsa(g, k, bound_cond)
        div = fvutils.vector_divergence(g)
        a = div * stress
        
        # Boundary conditions
        xf = g.face_centers
        u_bound = np.zeros((g.dim, g.num_faces))
        u_bound[0, bound_faces] = ux_f(xf[0, bound_faces], xf[1, bound_faces])
        u_bound[1, bound_faces] = uy_f(xf[0, bound_faces], xf[1, bound_faces])
        
        
        # Right hand side - contribution from the solution and the boundary conditions
        xc = g.cell_centers
        rhs = np.vstack((rhs_x_f(xc[0], xc[1]), rhs_y_f(xc[0], xc[1]))) * g.cell_volumes
        b = rhs.ravel('F') - div * bound_stress * u_bound.ravel('F')

        # Solve system, derive fluxes
        u_num = spsolve(a, b)
        stress_num = stress * u_num + bound_stress * u_bound.ravel('F')

        ux_num = u_num[::2]
        uy_num = u_num[1::2]
        
        stress_x_num = stress_num[::2]
        stress_y_num = stress_num[1::2]
        # Exact solution
        ux_ex = ux_f(xc[0], xc[1])
        uy_ex = uy_f(xc[0], xc[1])
        u_ex = np.vstack((ux_ex, uy_ex))
        u_diff = np.vstack((ux_num - ux_ex, uy_num - uy_ex))
        
        sx_ex_faces = np.vstack((sxx_f(xf[0], xf[1]), sxy_f(xf[0], xf[1])))
        sy_ex_faces = np.vstack((syx_f(xf[0], xf[1]), syy_f(xf[0], xf[1])))
        
        stress_x_ex = np.sum(g.face_normals[:2] * sx_ex_faces, axis=0)
        stress_y_ex = np.sum(g.face_normals[:2] * sy_ex_faces, axis=0)
        stress_diff = np.vstack((stress_x_num - stress_x_ex, 
                                 stress_y_num - stress_y_ex))
        stress_ex = np.vstack((stress_x_ex, stress_y_ex))
        
        u_err.append(np.sqrt(np.sum(g.cell_volumes * u_diff**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * stress_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * stress_ex**2)))
    return u_err, flux_err


grids = ['cart', 'triangular']


u_cart_nopert, f_cart_nopert = run_convergence('cart', 0)
print('Cartesian errors - no perturbations')
print(u_cart_nopert,'displacement error')
print(f_cart_nopert,'stress error')

u_cart_pert, f_cart_pert = run_convergence('cart', 0.5)
print('Cartesian errors - perturbation 0.5')
print(u_cart_pert,'displacement error')
print(f_cart_pert,'stress error')


u_triang_nopert, f_triang_nopert = run_convergence('triangular', 0)
print('Triangular errors - no perturbations')
print(u_triang_nopert,'displacement error')
print(f_triang_nopert,'stress error')


u_triang_pert, f_triang_pert = run_convergence('triangular', 0.5)
print('Triangular errors - perturbation 0.5')
print(u_triang_pert,'displacement error')
print(f_triang_pert,'stress error')

# EK: These values were hard-coded 12.06.2017.
assert np.abs(u_cart_nopert[2] - 0.00078374466288952144) < 1e-10
assert np.abs(f_cart_nopert[2] - 0.0030273898153929924) < 1e-10
assert np.abs(u_cart_pert[2] - 0.0010317052894504846) < 1e-10
assert np.abs(f_cart_pert[2] - 0.0053879362435559708) < 1e-10

assert np.abs(u_triang_nopert[2] - 0.0013462735517453231) < 1e-10
assert np.abs(f_triang_nopert[2] - 0.0084457184274578968) < 1e-10
assert np.abs(u_triang_pert[2] - 0.0015755518569344586) < 1e-10
assert np.abs(f_triang_pert[2] - 0.012835670048539171) < 1e-10

Cartesian errors - no perturbations
[0.011010878448828448, 0.0030197500225270933, 0.0007837446628904635, 0.00019841406340054548] displacement error
[0.036391713833312002, 0.010752498606472382, 0.0030273898153932912, 0.00084079909677677955] stress error
Cartesian errors - perturbation 0.5
[0.013141424612606898, 0.003959554975380245, 0.001031705289450405, 0.00024384620727654387] displacement error
[0.036187285790495613, 0.015764291256799894, 0.0053879362435557184, 0.0020697006664011576] stress error
Triangular errors - no perturbations
[0.017254389765132352, 0.0050218128535590217, 0.0013462735517447456, 0.0003461664448910652] displacement error
[0.082093297346373717, 0.027367216714651338, 0.0084457184274555879, 0.002619574268464812] stress error
Triangular errors - perturbation 0.5
[0.019724399988563231, 0.006135504619471847, 0.0015755518569345677, 0.00041613048050695065] displacement error
[0.077358151497225636, 0.030932401116355441, 0.012835670048538793, 0.0051826294978407558] stress e

# Neumann boundary conditions


In [7]:
# Permeability tensor, scalar for simplicity
mu = 1
lmbda = 1

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 3
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 3
grid_type = 'cart'

# Analytical solution
x, y = sympy.symbols('x y')
ux = (0.5-y)**2 * sympy.sin(y) * sympy.cos(y)
uy = y * (y-1) * sympy.cosh(x)
ux_f = sympy.lambdify((x, y), ux, 'numpy')
uy_f = sympy.lambdify((x, y), uy, 'numpy')
dux_x = sympy.diff(ux, x)
dux_y = sympy.diff(ux, y)
duy_x = sympy.diff(uy, x)
duy_y = sympy.diff(uy, y)
divu = dux_x + duy_y

sxx = 2 * mu * dux_x + lmbda * divu
sxy = mu * (dux_y + duy_x)
syx = mu * (duy_x + dux_y)
syy = 2 * mu * duy_y + lmbda * divu

sxx_f = sympy.lambdify((x, y), sxx, 'numpy')
sxy_f = sympy.lambdify((x, y), sxy, 'numpy')
syx_f = sympy.lambdify((x, y), syx, 'numpy')
syy_f = sympy.lambdify((x, y), syy, 'numpy')

rhs_x = sympy.diff(sxx, x) + sympy.diff(syx, y)
rhs_y = sympy.diff(sxy, x) + sympy.diff(syy, y)
rhs_x_f = sympy.lambdify((x, y), rhs_x, 'numpy')
rhs_y_f = sympy.lambdify((x, y), rhs_y, 'numpy')

def run_convergence(grid_type, pert):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert, domain=domain):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
            
        xf = g.face_centers
        sx_ex_faces = np.vstack((sxx_f(xf[0], xf[1]), sxy_f(xf[0], xf[1])))
        sy_ex_faces = np.vstack((syx_f(xf[0], xf[1]), syy_f(xf[0], xf[1])))
        
        stress_x_ex = np.sum(g.face_normals[:2] * sx_ex_faces, axis=0)
        stress_y_ex = np.sum(g.face_normals[:2] * sy_ex_faces, axis=0)

        # Permeability tensor
        mu_c = mu * np.ones(g.num_cells)
        lmbda_c = lmbda * np.ones(g.num_cells)
        k = stiffness(2, mu_c, lmbda_c)

        # Set type of boundary conditions - Dirichlet
        n = g.nodes.max()
        # Set type of boundary conditions - Dirichlet
        top = np.ravel(np.argwhere(g.face_centers[1, :] > n - 1e-10))
        bot = np.ravel(np.argwhere(g.face_centers[1, :] < 1e-10))
        left = np.ravel(np.argwhere(g.face_centers[0, :] < 1e-10))
        right = np.ravel(np.argwhere(g.face_centers[0, :] > n - 1e-10))

        dir_faces = np.hstack((bot, top))
        neu_faces = np.hstack((left, right))
        bound_cond = bc.BoundaryCondition(g, dir_faces, ['dir'] * dir_faces.size)
        
        # MPFA discretization, and system matrix
        stress, bound_stress = mpsa.mpsa(g, k, bound_cond)
        div = fvutils.vector_divergence(g)
        a = div * stress
        xf = g.face_centers
        
        nfi, _, sgn = sps.find(g.cell_faces[neu_faces,:])
        
        u_bound = np.zeros((g.dim, g.num_faces))
        u_bound[0, dir_faces] = ux_f(xf[0, dir_faces], xf[1, dir_faces])
        u_bound[1, dir_faces] = uy_f(xf[0, dir_faces], xf[1, dir_faces])
        
        # When setting the Neumann boundary condition we need to flip the 
        # sign of the faces with a normal pointing inwards as it is assumed
        # that the Neumann condition is the force from the boundary on to face
        u_bound[0, neu_faces[nfi]] = stress_x_ex[neu_faces[nfi]] * (sgn)
        u_bound[1, neu_faces[nfi]] = stress_y_ex[neu_faces[nfi]] * (sgn)
        # Right hand side - contribution from the solution and the boundary conditions
        xc = g.cell_centers
        rhs = np.vstack((rhs_x_f(xc[0], xc[1]), rhs_y_f(xc[0], xc[1]))) * g.cell_volumes
        b = rhs.ravel('F') - div * bound_stress * u_bound.ravel('F')

        # Solve system, derive fluxes
        u_num = spsolve(a, b)
        stress_num = stress * u_num + bound_stress * u_bound.ravel('F')

        ux_num = u_num[::2]
        uy_num = u_num[1::2]
        
        stress_x_num = stress_num[::2]
        stress_y_num = stress_num[1::2]
        # Exact solution
        ux_ex = ux_f(xc[0], xc[1])
        uy_ex = uy_f(xc[0], xc[1])
        u_ex = np.vstack((ux_ex, uy_ex))
        u_diff = np.vstack((ux_num - ux_ex, uy_num - uy_ex))
        
        stress_diff = np.vstack((stress_x_num - stress_x_ex, 
                                 stress_y_num - stress_y_ex))
        stress_ex = np.vstack((stress_x_ex, stress_y_ex))
        
        u_err.append(np.sqrt(np.sum(g.cell_volumes * u_diff**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * stress_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * stress_ex**2)))
                
    return u_err, flux_err

u_cart_nopert, f_cart_nopert = run_convergence('cart', 0)
print('Cartesian errors - no perturbations')
print(u_cart_nopert,'displacement error')
print(f_cart_nopert,'stress error')

u_triang_nopert, f_triang_nopert = run_convergence('triangular', 0)
print('Triangular errors - no perturbations')
print(u_triang_nopert,'displacement error')
print(f_triang_nopert,'stress error')

# # EK: These values were hard-coded 12.06.2017.
assert np.abs(u_cart_nopert[2] - 0.01978509916668196) < 1e-10
assert np.abs(f_cart_nopert[2] - 0.017633923832997497) < 1e-10

assert np.abs(u_triang_nopert[2] - 0.02009537946226712) < 1e-10
assert np.abs(f_triang_nopert[2] - 0.015470274728154886) < 1e-10

Cartesian errors - no perturbations
[0.19341322457532625, 0.053986061737524682, 0.01978509916668196] displacement error
[0.030804485699716087, 0.02272790742200646, 0.017633923832997497] stress error
Triangular errors - no perturbations
[0.22718530521895969, 0.062078841079084769, 0.02009537946226712] displacement error
[0.052949299769888859, 0.026954708583765235, 0.015470274728154886] stress error


We note that the reduction rates are smallar than for the pure Dirichlet case - the asymptotic rates are 1 for displacement and 0.5 for stress. The reason for this is not understood - there are no obvious bugs in the code. Todo..

In [29]:
###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 3
pert = 0.5

kappa_list = [1e-6, 1, 1e6]
### End of parameter definitions

# Permeability tensor, scalar for simplicity
mu = 1
mbda = 1

def chi(x, y):
    return np.logical_and(np.greater(x, 0.5), np.greater(y, 0.5))

# Analytical solution
x, y = sympy.symbols('x y')
ux = sympy.sin(2*pi * x) * sympy.sin(2 * pi * y)
uy = sympy.cos(pi * x) * (y-0.5)**2
ux_f = sympy.lambdify((x, y), ux, 'numpy')
uy_f = sympy.lambdify((x, y), uy, 'numpy')
dux_x = sympy.diff(ux, x)
dux_y = sympy.diff(ux, y)
duy_x = sympy.diff(uy, x)
duy_y = sympy.diff(uy, y)
divu = dux_x + duy_y

sxx = 2 * mu * dux_x + lmbda * divu
sxy = mu * (dux_y + duy_x)
syx = mu * (duy_x + dux_y)
syy = 2 * mu * duy_y + lmbda * divu

sxx_f = sympy.lambdify((x, y), sxx, 'numpy')
sxy_f = sympy.lambdify((x, y), sxy, 'numpy')
syx_f = sympy.lambdify((x, y), syx, 'numpy')
syy_f = sympy.lambdify((x, y), syy, 'numpy')

rhs_x = sympy.diff(sxx, x) + sympy.diff(syx, y)
rhs_y = sympy.diff(sxy, x) + sympy.diff(syy, y)
rhs_x_f = sympy.lambdify((x, y), rhs_x, 'numpy')
rhs_y_f = sympy.lambdify((x, y), rhs_y, 'numpy')


def run_convergence(grid_type, pert):
    u_err = np.zeros((num_refs, len(kappa_list)))
    flux_err = np.copy(u_err)
    
    for iter1, g in enumerate(setup_grids.grid_sequence_fixed_lines(basedim, num_refs, grid_type, pert, subdom_func=chi)):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        for iter2, kappa in enumerate(kappa_list):        
        # Permeability tensor
            char_func_cells = chi(g.cell_centers[0], g.cell_centers[1]) * 1.
            mat_vec = (1 - char_func_cells) + kappa * char_func_cells

            k = stiffness(2, mat_vec, mat_vec)

            # Set type of boundary conditions - Dirichlet
            bound_faces = g.get_all_boundary_faces()
            bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)

            # MPFA discretization, and system matrix
            stress, bound_stress = mpsa.mpsa(g, k, bound_cond)
            div = fvutils.vector_divergence(g)
            a = div * stress

            # Boundary conditions
            xf = g.face_centers
            char_func_bound = chi(xf[0, bound_faces], xf[1, bound_faces]) * 1
            u_bound = np.zeros((g.dim, g.num_faces))
            u_bound[0, bound_faces] = ux_f(xf[0, bound_faces], xf[1, bound_faces])\
                    / ((1 - char_func_bound) + kappa * char_func_bound)
            u_bound[1, bound_faces] = uy_f(xf[0, bound_faces], xf[1, bound_faces]) \
                    / ((1 - char_func_bound) + kappa * char_func_bound)

            # Right hand side - contribution from the solution and the boundary conditions
            xc = g.cell_centers
            rhs = np.vstack((rhs_x_f(xc[0], xc[1]), 
                             rhs_y_f(xc[0], xc[1]))) * g.cell_volumes
            b = rhs.ravel('F') - div * bound_stress * u_bound.ravel('F')

            # Solve system, derive fluxes
            u_num = spsolve(a, b)
            stress_num = stress * u_num + bound_stress * u_bound.ravel('F')

            ux_num = u_num[::2]
            uy_num = u_num[1::2]
            
            stress_x_num = stress_num[::2]
            stress_y_num = stress_num[1::2]
            # Exact solution
            ux_ex = ux_f(xc[0], xc[1])/ ((1 - char_func_cells) + kappa * char_func_cells)
            uy_ex = uy_f(xc[0], xc[1])/ ((1 - char_func_cells) + kappa * char_func_cells)
            u_ex = np.vstack((ux_ex, uy_ex))
            u_diff = np.vstack((ux_num - ux_ex, uy_num - uy_ex))

            sx_ex_faces = np.vstack((sxx_f(xf[0], xf[1]), sxy_f(xf[0], xf[1])))
            sy_ex_faces = np.vstack((syx_f(xf[0], xf[1]), syy_f(xf[0], xf[1])))

            stress_x_ex = np.sum(g.face_normals[:2] * sx_ex_faces, axis=0)
            stress_y_ex = np.sum(g.face_normals[:2] * sy_ex_faces, axis=0)
            stress_diff = np.vstack((stress_x_num - stress_x_ex, 
                                     stress_y_num - stress_y_ex))
            stress_ex = np.vstack((stress_x_ex, stress_y_ex))
            
            u_err[iter1, iter2] = np.sqrt(np.sum(g.cell_volumes * u_diff**2)) \
                        / np.sqrt(np.sum(g.cell_volumes * u_ex**2))
            flux_err[iter1, iter2] = np.sqrt(np.sum((g.face_areas ** g.dim) * stress_diff**2))\
                        / np.sqrt(np.sum((g.face_areas ** g.dim) * stress_ex**2))

    return u_err, flux_err

u_cart_nopert, f_cart_nopert = run_convergence('cart', 0)
print('Cartesian errors - no perturbations')
print(u_cart_nopert,'displacement error')
print(f_cart_nopert,'stress error')

u_cart_pert, f_cart_pert = run_convergence('cart', 0.5)
print('Cartesian errors - perturbation 0.5')
print(u_cart_pert,'displacement error')
print(f_cart_pert,'stress error')


u_triang_nopert, f_triang_nopert = run_convergence('triangular', 0)
print('Triangular errors - no perturbations')
print(u_triang_nopert,'displacement error')
print(f_triang_nopert,'stress error')


u_triang_pert, f_triang_pert = run_convergence('triangular', 0.5)
print('Triangular errors - perturbation 0.5')
print(u_triang_pert,'displacement error')
print(f_triang_pert,'stress error')

# EK: These values were hard-coded 12.06.2017.
assert np.abs(u_cart_nopert[2, 0] - 0.01980355  ) < 1e-7
assert np.abs(f_cart_nopert[2, 0] - 0.02181871   ) < 1e-7
assert np.abs(u_cart_pert[2, 0] - 0.02294892  ) < 1e-7
assert np.abs(f_cart_pert[2, 0] - 0.02941291  ) < 1e-7

assert np.abs(u_triang_nopert[2, 0] - 0.01448248  ) < 1e-7
assert np.abs(f_triang_nopert[2, 0] - 0.02437554  ) < 1e-7
assert np.abs(u_triang_pert[2, 0] - 0.01892896  ) < 1e-7
assert np.abs(f_triang_pert[2, 0] - 0.0326783   ) < 1e-7

Cartesian errors - no perturbations
[[ 0.37388856  0.4014268   0.38285806]
 [ 0.08186136  0.09736873  0.08866018]
 [ 0.01980355  0.02491816  0.02207893]] displacement error
[[ 0.16987677  0.16287318  0.17095043]
 [ 0.06445937  0.05898811  0.06587345]
 [ 0.02181871  0.01941263  0.02240798]] stress error
Cartesian errors - perturbation 0.5
[[ 0.38707647  0.51439666  0.52211052]
 [ 0.10514942  0.11284306  0.10794662]
 [ 0.02294892  0.02818847  0.02665215]] displacement error
[[ 0.22785554  0.22643673  0.23802875]
 [ 0.07617105  0.06993615  0.07181565]
 [ 0.02941291  0.02754268  0.02920611]] stress error
Triangular errors - no perturbations
[[ 0.17333004  0.23479125  0.20444783]
 [ 0.05026037  0.07676565  0.0660371 ]
 [ 0.01448248  0.02256     0.01928829]] displacement error
[[ 0.15556813  0.14600391  0.15354293]
 [ 0.06803554  0.06303363  0.06653757]
 [ 0.02437554  0.02244595  0.02451626]] stress error
Triangular errors - perturbation 0.5
[[ 0.19496202  0.37736155  0.37082897]
 [ 0.063949

# Incompressible limit

In [42]:
# Permeability tensor, scalar for simplicity
mu = 1
lmbda = 1

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 4
grid_type = 'cart'

# Analytical solution
x, y = sympy.symbols('x y')
ux = sympy.sin(y) * (sympy.cos(x)-1)
uy = sympy.sin(x) * (1-sympy.cos(y))
ux_f = sympy.lambdify((x, y), ux, 'numpy')
uy_f = sympy.lambdify((x, y), uy, 'numpy')
dux_x = sympy.diff(ux, x)
dux_y = sympy.diff(ux, y)
duy_x = sympy.diff(uy, x)
duy_y = sympy.diff(uy, y)
divu = dux_x + duy_y

sxx = 2 * mu * dux_x + lmbda * divu
sxy = mu * (dux_y + duy_x)
syx = mu * (duy_x + dux_y)
syy = 2 * mu * duy_y + lmbda * divu

sxx_f = sympy.lambdify((x, y), sxx, 'numpy')
sxy_f = sympy.lambdify((x, y), sxy, 'numpy')
syx_f = sympy.lambdify((x, y), syx, 'numpy')
syy_f = sympy.lambdify((x, y), syy, 'numpy')

rhs_x = sympy.diff(sxx, x) + sympy.diff(syx, y)
rhs_y = sympy.diff(sxy, x) + sympy.diff(syy, y)
rhs_x_f = sympy.lambdify((x, y), rhs_x, 'numpy')
rhs_y_f = sympy.lambdify((x, y), rhs_y, 'numpy')

def run_convergence(grid_type, pert):
    u_err = np.zeros((num_refs, 3))
    flux_err = np.zeros((num_refs, 3))

    for iter1, g in enumerate(setup_grids.grid_sequence(basedim, num_refs, grid_type, pert, domain=domain)):
        for iter2, exponent in enumerate(range(2, 5)):
            # Reset the random seed for every grid realization.
            # This should make no difference for the convergence test, 
            # but it makes sure that we can run unit tests based on the values obtained
            # here.
            np.random.seed(42)

            lmbda = 10**exponent

            # Permeability tensor
            mu_c = mu * np.ones(g.num_cells)
            lmbda_c = lmbda * np.ones(g.num_cells)
            k = stiffness(2, mu_c, lmbda_c)

            # Set type of boundary conditions - Dirichlet
            bound_faces = g.get_all_boundary_faces()
            bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)

            # MPFA discretization, and system matrix
            stress, bound_stress = mpsa.mpsa(g, k, bound_cond)
            div = fvutils.vector_divergence(g)
            a = div * stress

            # Boundary conditions
            xf = g.face_centers
            u_bound = np.zeros((g.dim, g.num_faces))
            u_bound[0, bound_faces] = ux_f(xf[0, bound_faces], xf[1, bound_faces])
            u_bound[1, bound_faces] = uy_f(xf[0, bound_faces], xf[1, bound_faces])


            # Right hand side - contribution from the solution and the boundary conditions
            xc = g.cell_centers
            rhs = np.vstack((rhs_x_f(xc[0], xc[1]), rhs_y_f(xc[0], xc[1]))) * g.cell_volumes
            b = rhs.ravel('F') - div * bound_stress * u_bound.ravel('F')

            # Solve system, derive fluxes
            u_num = spsolve(a, b)
            stress_num = stress * u_num + bound_stress * u_bound.ravel('F')

            ux_num = u_num[::2]
            uy_num = u_num[1::2]

            stress_x_num = stress_num[::2]
            stress_y_num = stress_num[1::2]
            # Exact solution
            ux_ex = ux_f(xc[0], xc[1])
            uy_ex = uy_f(xc[0], xc[1])
            u_ex = np.vstack((ux_ex, uy_ex))
            u_diff = np.vstack((ux_num - ux_ex, uy_num - uy_ex))

            sx_ex_faces = np.vstack((sxx_f(xf[0], xf[1]), sxy_f(xf[0], xf[1])))
            sy_ex_faces = np.vstack((syx_f(xf[0], xf[1]), syy_f(xf[0], xf[1])))

            stress_x_ex = np.sum(g.face_normals[:2] * sx_ex_faces, axis=0)
            stress_y_ex = np.sum(g.face_normals[:2] * sy_ex_faces, axis=0)
            stress_diff = np.vstack((stress_x_num - stress_x_ex, 
                                     stress_y_num - stress_y_ex))
            stress_ex = np.vstack((stress_x_ex, stress_y_ex))
        
            u_err[iter1, iter2] = np.sqrt(np.sum(g.cell_volumes * u_diff**2)) \
                        / np.sqrt(np.sum(g.cell_volumes * u_ex**2))
            flux_err[iter1, iter2] = np.sqrt(np.sum((g.face_areas ** g.dim) * stress_diff**2))\
                        / np.sqrt(np.sum((g.face_areas ** g.dim) * stress_ex**2))

    return u_err, flux_err

u_cart_nopert, f_cart_nopert = run_convergence('cart', 0)
print('Cartesian errors - no perturbations')
print(u_cart_nopert,'displacement error')
print(f_cart_nopert,'stress error')

u_cart_pert, f_cart_pert = run_convergence('cart', 0.5)
print('Cartesian errors - perturbation 0.5')
print(u_cart_pert,'displacement error')
print(f_cart_pert,'stress error')


u_triang_nopert, f_triang_nopert = run_convergence('triangular', 0)
print('Triangular errors - no perturbations')
print(u_triang_nopert,'displacement error')
print(f_triang_nopert,'stress error')


u_triang_pert, f_triang_pert = run_convergence('triangular', 0.5)
print('Triangular errors - perturbation 0.5')
print(u_triang_pert,'displacement error')
print(f_triang_pert,'stress error')

# EK: These values were hard-bcoded 12.06.2017.
assert np.abs(u_cart_nopert[2, 0] -  0.00248737   ) < 1e-7
assert np.abs(f_cart_nopert[2, 0] -    0.00646923  ) < 1e-7
assert np.abs(u_cart_pert[2, 0] -  0.00270644   ) < 1e-7
assert np.abs(f_cart_pert[2, 0] -  0.01322056   ) < 1e-7

assert np.abs(u_triang_nopert[2, 0] -  0.00194895   ) < 1e-7
assert np.abs(f_triang_nopert[2, 0] -   0.00479015  ) < 1e-7
assert np.abs(u_triang_pert[2, 0] -   0.0028185   ) < 1e-7
assert np.abs(f_triang_pert[2, 0] -   1.46222072e-02 ) < 1e-7

Cartesian errors - no perturbations
[[ 0.03697053  0.03753405  0.03759228]
 [ 0.00976093  0.00987151  0.00988294]
 [ 0.00248737  0.00251144  0.00251393]
 [ 0.00062543  0.00063108  0.00063166]] displacement error
[[ 0.05751235  0.05696182  0.05690496]
 [ 0.0198125   0.01970924  0.01969853]
 [ 0.00646923  0.00645496  0.00645348]
 [ 0.00209802  0.00209683  0.0020967 ]] stress error
Cartesian errors - perturbation 0.5
[[ 0.03956063  0.04032235  0.04040406]
 [ 0.01064916  0.01079559  0.01081118]
 [ 0.00270644  0.00277624  0.00279648]
 [ 0.0007012   0.00071505  0.00072281]] displacement error
[[ 0.10770594  0.54796429  5.20910705]
 [ 0.03973921  0.24607287  2.44840797]
 [ 0.01322056  0.02545624  0.19096983]
 [ 0.00524277  0.0151903   0.1813106 ]] stress error
Triangular errors - no perturbations
[[ 0.02904832  0.02928625  0.02931099]
 [ 0.00756921  0.00760437  0.00760814]
 [ 0.00194895  0.00195491  0.00195557]
 [ 0.00049507  0.00049632  0.00049647]] displacement error
[[ 0.04114073  0.041005

Note that on triangular grids, the convergence order for stress deteriorates for high Poisson's ratios. See Keilegavlen, Nordbotten IJNME 2017 for details.

# 3d homogeneous problem

This example also compares computational times for discretization and two linear solvers (direct and amg)

In [9]:
# Permeability tensor, scalar for simplicity
mu = 1
lmbda = 1

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 2
domain = np.array([1, 1, 1])
basedim = np.array([base, base, base])
num_refs = 3
grid_type = 'cart'

# Analytical solution. Essentially randomly made
x, y, z = sympy.symbols('x y z')
ux = sympy.sin(x) * sympy.cos(y) * (0.5 - z)**2
uy = sympy.sin(y) * x**2 * sympy.cos(z)
uz = sympy.cos(x) * sympy.sin(z) * y * (1-z)
ux_f = sympy.lambdify((x, y, z), ux, 'numpy')
uy_f = sympy.lambdify((x, y, z), uy, 'numpy')
uz_f = sympy.lambdify((x, y, z), uz, 'numpy')
dux_x = sympy.diff(ux, x)
dux_y = sympy.diff(ux, y)
dux_z = sympy.diff(ux, z)
duy_x = sympy.diff(uy, x)
duy_y = sympy.diff(uy, y)
duy_z = sympy.diff(uy, z)
duz_x = sympy.diff(uz, x)
duz_y = sympy.diff(uz, y)
duz_z = sympy.diff(uz, z)

divu = dux_x + duy_y + duz_z

sxx = 2 * mu * dux_x + lmbda * divu
sxy = mu * (dux_y + duy_x)
sxz = mu * (dux_z + duz_x) 
syx = mu * (duy_x + dux_y)
syy = 2 * mu * duy_y + lmbda * divu
syz = mu * (duy_z + duz_y)
szx = mu * (duz_x + dux_z)
szy = mu * (duy_z + duz_y)
szz = 2 * mu * duz_z + lmbda * divu

sxx_f = sympy.lambdify((x, y, z), sxx, 'numpy')
sxy_f = sympy.lambdify((x, y, z), sxy, 'numpy')
sxz_f = sympy.lambdify((x, y, z), sxz, 'numpy')
syx_f = sympy.lambdify((x, y, z), syx, 'numpy')
syy_f = sympy.lambdify((x, y, z), syy, 'numpy')
syz_f = sympy.lambdify((x, y, z), syz, 'numpy')
szx_f = sympy.lambdify((x, y, z), szx, 'numpy')
szy_f = sympy.lambdify((x, y, z), szy, 'numpy')
szz_f = sympy.lambdify((x, y, z), szz, 'numpy')

rhs_x = sympy.diff(sxx, x) + sympy.diff(syx, y) + sympy.diff(szx, z)
rhs_y = sympy.diff(sxy, x) + sympy.diff(syy, y) + sympy.diff(szy, z)
rhs_z = sympy.diff(sxz, x) + sympy.diff(syz, y) + sympy.diff(szz, z)
rhs_x_f = sympy.lambdify((x, y, z), rhs_x, 'numpy')
rhs_y_f = sympy.lambdify((x, y, z), rhs_y, 'numpy')
rhs_z_f = sympy.lambdify((x, y, z), rhs_z, 'numpy')


def run_convergence(grid_type, pert):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert, domain=domain):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
            
        # Permeability tensor
        mu_c = mu * np.ones(g.num_cells)
        lmbda_c = lmbda * np.ones(g.num_cells)
        k = stiffness(g.dim, mu_c, lmbda_c)

        # Set type of boundary conditions - Dirichlet
        bound_faces = g.get_boundary_faces()
        bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)
        
        # MPFA discretization, and system matrix
        tm = time()
        stress, bound_stress = mpsa.mpsa(g, k, bound_cond)
        print('Time for discretization ' + str(time() - tm))
        div = fvutils.vector_divergence(g)
        a = div * stress
        a.tocsr()
        
        # Boundary conditions
        xf = g.face_centers
        u_bound = np.zeros((g.dim, g.num_faces))
        u_bound[0, bound_faces] = ux_f(xf[0, bound_faces], xf[1, bound_faces], xf[2, bound_faces])
        u_bound[1, bound_faces] = uy_f(xf[0, bound_faces], xf[1, bound_faces], xf[2, bound_faces])
        u_bound[2, bound_faces] = uz_f(xf[0, bound_faces], xf[1, bound_faces], xf[2, bound_faces])
        
        # Right hand side - contribution from the solution and the boundary conditions
        xc = g.cell_centers
        rhs = np.vstack((rhs_x_f(xc[0], xc[1], xc[2]), 
                         rhs_y_f(xc[0], xc[1], xc[2]),
                         rhs_z_f(xc[0], xc[1], xc[2]))) * g.cell_volumes
        b = rhs.ravel('F') - div * bound_stress * u_bound.ravel('F')

        # Solve system, derive fluxes
        tm = time()
        smooth = smooth=('energy', {'krylov': 'gmres', 'degree': 2})
        
        null_space = np.zeros((g.dim * g.num_cells, 6))
        null_space[0::g.dim, 0] = 1
        null_space[1::g.dim, 1] = 1
        null_space[2::g.dim, 2] = 1
        # Rotations
        null_space[0::g.dim, 3] = 1
        null_space[1::g.dim, 3] = -1
        null_space[1::g.dim, 4] = 1
        null_space[2::g.dim, 4] = -1
        null_space[2::g.dim, 5] = 1
        null_space[0::g.dim, 5] = -1

        res = []
        ml = pyamg.smoothed_aggregation_solver(a, null_space, smooth=smooth, symmetry='nonsymmetric')
        u_num = ml.solve(b, residuals=res, tol=1e-12)
        stress_num = stress * u_num + bound_stress * u_bound.ravel('F')
        print('Time for AMG ' + str(time() - tm))
        ux_num = u_num[::g.dim]
        uy_num = u_num[1::g.dim]
        uz_num = u_num[2::g.dim]
        
        stress_x_num = stress_num[::g.dim]
        stress_y_num = stress_num[1::g.dim]
        stress_z_num = stress_num[2::g.dim]
        # Exact solution
        ux_ex = ux_f(xc[0], xc[1], xc[2])
        uy_ex = uy_f(xc[0], xc[1], xc[2])
        uz_ex = uz_f(xc[0], xc[1], xc[2])
        u_ex = np.vstack((ux_ex, uy_ex, uz_ex))
        u_diff = np.vstack((ux_num - ux_ex, uy_num - uy_ex))
        
        sx_ex_faces = np.vstack((sxx_f(xf[0], xf[1], xf[2]),
                                 sxy_f(xf[0], xf[1], xf[2]),
                                 sxz_f(xf[0], xf[1], xf[2])))
        
        sy_ex_faces = np.vstack((syx_f(xf[0], xf[1], xf[2]),
                                 syy_f(xf[0], xf[1], xf[2]),
                                 syz_f(xf[0], xf[1], xf[2])))
        
        sz_ex_faces = np.vstack((szx_f(xf[0], xf[1], xf[2]),
                                 szy_f(xf[0], xf[1], xf[2]),
                                 szz_f(xf[0], xf[1], xf[2])))

        stress_x_ex = np.sum(g.face_normals[:g.dim] * sx_ex_faces, axis=0)
        stress_y_ex = np.sum(g.face_normals[:g.dim] * sy_ex_faces, axis=0)
        stress_z_ex = np.sum(g.face_normals[:g.dim] * sz_ex_faces, axis=0)
        stress_diff = np.vstack((stress_x_num - stress_x_ex, 
                                 stress_y_num - stress_y_ex,
                                 stress_z_num - stress_z_ex))
        stress_ex = np.vstack((stress_x_ex, stress_y_ex, stress_z_ex))
        
        u_err.append(np.sqrt(np.sum(g.cell_volumes * u_diff**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * stress_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * stress_ex**2)))
    return u_err, flux_err


grids = ['cart', 'triangular']


u_cart_nopert, f_cart_nopert = run_convergence('cart', 0)
print('Cartesian errors - no perturbations')
print(u_cart_nopert,'displacement error')
print(f_cart_nopert,'stress error')

u_cart_pert, f_cart_pert = run_convergence('cart', 0.5)
print('Cartesian errors - perturbation 0.5')
print(u_cart_pert,'displacement error')
print(f_cart_pert,'stress error')


u_triang_nopert, f_triang_nopert = run_convergence('tetrahedral', 0)
print('Triangular errors - no perturbations')
print(u_triang_nopert,'displacement error')
print(f_triang_nopert,'stress error')


u_triang_pert, f_triang_pert = run_convergence('tetrahedral', 0.3)
print('Triangular errors - perturbation 0.5')
print(u_triang_pert,'displacement error')
print(f_triang_pert,'stress error')

# EK: These values were hard-coded 16.06.2017.
assert np.abs(u_cart_nopert[2] - 0.0096234771484497109) < 1e-10
assert np.abs(f_cart_nopert[2] - 0.018777567804377917) < 1e-10
assert np.abs(u_cart_pert[2] - 0.0097853910627935265) < 1e-10
assert np.abs(f_cart_pert[2] - 0.024831033744125156) < 1e-10

assert np.abs(u_triang_nopert[2] - 0.0058583454307817992) < 1e-10
assert np.abs(f_triang_nopert[2] - 0.026474428723816987) < 1e-10
assert np.abs(u_triang_pert[2] - 0.0063364542707427755) < 1e-10
assert np.abs(f_triang_pert[2] - 0.028588515746948068) < 1e-10

Time for discretization 0.7849624156951904
Time for AMG 0.010245084762573242


/home/eke001/anaconda3/envs/conquer/lib/python3.5/site-packages/pyamg/aggregation/aggregation.py:214: SparseEfficiencyWarning: Implicit conversion of A to CSR
  SparseEfficiencyWarning)


Time for discretization 1.0785565376281738
Time for AMG 0.06386494636535645


/home/eke001/anaconda3/envs/conquer/lib/python3.5/site-packages/pyamg/aggregation/aggregation.py:214: SparseEfficiencyWarning: Implicit conversion of A to CSR
  SparseEfficiencyWarning)


Time for discretization 1.6758003234863281


...

Time for discretization 1.6282703876495361


/home/eke001/anaconda3/envs/conquer/lib/python3.5/site-packages/pyamg/aggregation/aggregation.py:214: SparseEfficiencyWarning: Implicit conversion of A to CSR
  SparseEfficiencyWarning)


Time for AMG 0.24346184730529785
Time for discretization 4.506616115570068


/home/eke001/anaconda3/envs/conquer/lib/python3.5/site-packages/pyamg/aggregation/aggregation.py:214: SparseEfficiencyWarning: Implicit conversion of A to CSR
  SparseEfficiencyWarning)


Time for AMG 3.131685495376587
Triangular errors - perturbation 0.5
[0.067581938666718508, 0.022520441856033087, 0.0063364542707427755] displacement error
[0.15530102937536167, 0.067045393676151102, 0.028588515746948068] stress error


In [5]:
pyamg.smoothed_aggregation_solver?